In [1]:
class PTS:
    # n is an integer > 1, constellation is an instance of the Constellation 
    # class on Sym(n)
    def __init__(self, n, constellation):
        self.n = n
        self.constellation = constellation
        self.orbit = None

    def __eq__(self, other):
        return (self.constellation == other.constellation)
    
    def __neq__(self, other):
        return (self.constellation != other.constellation)

    def __hash__(self):
        return self.constellation.__hash__()

    def g(self,i):
        return self.constellation.g(i)
    
    def braid_group_orbit(self):
        orbit = set()
        waiting = [self]

        while waiting:
            
            c = waiting.pop()
            orbit.add(c)
            for i in range(3): 
                cc = c.braid(i) 
                if cc not in orbit:
                    waiting.append(cc)
        return orbit
    
    # Will return false as soon as possible, sets orbit as state
    def braid_group_orbit_and_one_cyl_check(self):
        orbit = set()
        waiting = [self]

        while waiting:
            c = waiting.pop()
            orbit.add(c)
            for i in range(3):
                cc = c.braid(i) 
                if cc not in orbit:
                    if cc.check_horizontal_direction():
                        waiting.append(cc)
                    else:
                        return False
        
        self.orbit = orbit
        return True
    
    def iso_class_braid_group_orbit(self):
        orbit = []
        for t in self.constellation.braid_group_orbit().vertices():
            orbit.append(PTS(self.n, t))
        
        return orbit
    
    def braid(self, i):
        return PTS(self.n, self.constellation.braid_group_action(i))

    def is_cyclic(self):
        G = SymmetricGroup(self.n).subgroup([self.g(0), self.g(1), self.g(2), self.g(3)])
        return G.is_cyclic()
    
    def is_normal(self):
        G = SymmetricGroup(self.n).subgroup([self.g(0), self.g(1), self.g(2), self.g(3)])
        return G.is_normal() 
    
    # check if there is one cylinder in the horizontal direction
    def check_horizontal_direction(self):
        if (self.g(0)*self.g(1)).cycle_type() == [self.n]:
            return True 
        else:
            return False
    
    def is_one_cyl_surface(self):
        if self.orbit == None:
            self.orbit = self.braid_group_orbit()

        for t in self.orbit: # t not nec = pts, but will be isom. constellations
            if not PTS(self.n, t).check_horizontal_direction():
                return False
        return True
    
    def contains_identity(self):
        for i in range(0,4):
            if self.g(i).cycle_type() == ([1]*self.n):
                return True 
        return False

    def __str__(self):
        return f' ({self.g(0)},  {self.g(1)},  {self.g(2)},  {self.g(3)})'

    def verbose_print(self):
        print('PTS defined by:')
        print(f' a: {self.g(0)}')
        print(f' b: {self.g(1)}')
        print(f' c: {self.g(2)}')
        print(f' d: {self.g(3)}\n')
    
        print(f'cycle type: {self.constellation.profile()}')
        print(f'is cyclic: {self.is_cyclic()}')
        print(f'is normal: {self.is_normal()}')
        print(f'is 1-cylinder: {self.is_one_cyl_surface()}')
        print(f'Braid group orbit:')
        if self.orbit == None:
            self.orbit = self.braid_group_orbit()
        for t in self.orbit:
            print(t)
        print(f'\n')


In [2]:
from sage.all import SymmetricGroup

n = 7
Sn = SymmetricGroup(n)
characters = Sn.character_table()
conjugacy_classes = Sn.conjugacy_classes()
print(conjugacy_classes)
print(characters)

[Conjugacy class of cycle type [1, 1, 1, 1, 1, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [2, 1, 1, 1, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [2, 2, 1, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [2, 2, 2, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [3, 1, 1, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [3, 2, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [3, 2, 2] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [3, 3, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [4, 1, 1, 1] in Symmetric group of order 7! as a permutation group, Conjugacy class of cycle type [4, 2, 1] in Symmetric group of order 7! as a permutation group, Conjuga

In [3]:
def frob(n, class_indices):
    Sn = SymmetricGroup(n)
    characters = Sn.character_table()
    k = len(Sn.conjugacy_classes())

    sum = 0
    for i in range(0, k):
        chi = characters[i]
        product = chi[0]^2
        for i in class_indices:
            product *= (chi[i]/chi[0])
        sum += product

    return sum


In [4]:
# these are the conjugacy classes containing a and b such that ab is an n-cycle.
conj_class_pairs = set()
k = len(Sn.conjugacy_classes())

# if ab is an n-cycle, then ab^-1= c is too. so abc = id.
# we're looking for (a,b) s.t. abc = id for some c an n-cycle.
for i in range(0,len(conjugacy_classes)):
    for j in range(i, len(conjugacy_classes)):
        if frob(n, [i,j,k-1]) != 0:
            conj_class_pairs.add((i,j))

print(len(conj_class_pairs))
# TODO: these numbers seem very small, and dividing by |G| doesn't make sense

50


In [5]:
quads = set()

for (a,b) in conj_class_pairs:
    for i in range(len(conjugacy_classes)):
        if tuple(sorted([a,i])) in conj_class_pairs and tuple(sorted([b,i])) in conj_class_pairs:
            for j in range(len(conjugacy_classes)):
                if tuple(sorted([a,j])) in conj_class_pairs and tuple(sorted([b,j])) in conj_class_pairs and tuple(sorted([i,j])) in conj_class_pairs:
                    quads.add(tuple(sorted([a,b,i,j])))

print(len(quads))   
print(quads)         


277
{(5, 10, 10, 13), (6, 6, 7, 11), (3, 3, 5, 13), (5, 8, 13, 13), (4, 6, 9, 9), (3, 5, 5, 12), (6, 7, 7, 11), (6, 11, 11, 11), (3, 3, 10, 12), (5, 5, 13, 13), (10, 12, 13, 13), (3, 5, 13, 13), (5, 10, 12, 13), (8, 12, 13, 13), (11, 11, 14, 14), (10, 10, 13, 13), (6, 9, 9, 11), (4, 7, 14, 14), (1, 10, 10, 10), (12, 12, 12, 13), (5, 8, 10, 13), (3, 3, 12, 12), (8, 10, 12, 13), (3, 13, 13, 13), (9, 9, 14, 14), (3, 5, 5, 5), (4, 6, 7, 14), (4, 6, 9, 11), (3, 8, 12, 13), (3, 3, 8, 8), (3, 5, 8, 10), (6, 6, 11, 14), (2, 9, 11, 14), (4, 6, 11, 11), (5, 5, 10, 10), (1, 10, 10, 12), (11, 11, 11, 11), (5, 12, 12, 12), (4, 6, 7, 7), (4, 7, 11, 11), (6, 11, 14, 14), (8, 8, 8, 10), (2, 6, 14, 14), (3, 3, 8, 10), (3, 5, 8, 12), (1, 10, 12, 12), (8, 13, 13, 13), (5, 5, 5, 13), (9, 9, 11, 11), (6, 6, 6, 14), (3, 5, 12, 12), (8, 8, 12, 13), (8, 12, 12, 12), (5, 5, 10, 12), (2, 6, 11, 14), (8, 8, 10, 10), (6, 9, 14, 14), (4, 6, 7, 9), (4, 11, 11, 14), (3, 10, 10, 13), (3, 3, 8, 12), (2, 7, 11, 14), (5

In [8]:
for (i,q) in enumerate(quads):
    if q == (0,14,14,14):
        print(i)
    elif q == (14, 14, 14, 14):
        print(i)

102
256


In [ ]:
# quads[0] takes 2m 27s 
# quads[2] takes 20s
# computed: 0-10 102

candidate_tuples = []

for (c1, c2, c3, c4) in [list(quads)[2]]:
    for a in conjugacy_classes[c1]:
        for b in conjugacy_classes[c2]:
            if (a*b).cycle_type() == [n]:
                for c in conjugacy_classes[c3]:
                    if (a*c).cycle_type() == [n] and (b*c).cycle_type() == [n]:
                        d = c.inverse()*b.inverse()*a.inverse()
                        pts = [a,b,c,d]
                        candidate_tuples.append(pts) 
print(len(candidate_tuples))  

277
211680


In [ ]:
candidate_tuples = []

for (c1, c2, c3, c4) in list(quads)[10:14]:
    for a in conjugacy_classes[c1]:
        for b in conjugacy_classes[c2]:
            if (a*b).cycle_type() == [n]:
                for c in conjugacy_classes[c3]:
                    if (a*c).cycle_type() == [n] and (b*c).cycle_type() == [n]:
                        d = c.inverse()*b.inverse()*a.inverse()
                        pts = [a,b,c,d]
                        candidate_tuples.append(pts) 
print(len(candidate_tuples))  

from sage.combinat import constellation

orbit_reps = set()
orbit_computed = set()
reps_computed = 0 # for tracking computation progress
computed = 0
checkpoint = 0

for pts in candidate_tuples:
    p = PTS(n, Constellation(pts))
    if p in orbit_computed:
        computed +=1
        continue 
    else:
        # print('found a new orbit rep, computing orbit')
        is_one_cylinder = p.braid_group_orbit_and_one_cyl_check()
        if is_one_cylinder:
            for o in p.orbit:
                orbit_computed.add(o)
                computed += 1
        
            orbit_reps.add(p)
            reps_computed += 1
        else:
            orbit_computed.add(p)
            computed += 1

        # print(f'orbit reps found: {reps_computed}')
    
    if len(orbit_computed) >= 10000000:
        break

print(len(orbit_reps))
    

4284000
0


In [13]:
print(len(orbit_computed))
print(len(orbit_reps))
one_cyl_orbit_reps = set()

for pts in orbit_reps:
    if pts.is_one_cyl_surface():
        one_cyl_orbit_reps.add(pts)

print(len(one_cyl_orbit_reps))

876960
0
0


In [11]:
one_cyl_orbit_reps_cycle_types = {}
for pts in one_cyl_orbit_reps:
    key = tuple(sorted(pts.constellation.profile()))
    if not one_cyl_orbit_reps_cycle_types.get(key)==None:
        one_cyl_orbit_reps_cycle_types.get(key).append(pts)
    else:
        one_cyl_orbit_reps_cycle_types.update({key: [pts]})
print(f'Number of cycle types: {len(one_cyl_orbit_reps_cycle_types.keys())}')
for key in one_cyl_orbit_reps_cycle_types.keys():
    print(f'{len(one_cyl_orbit_reps_cycle_types.get(key))} orbit reps of type {key}')

Number of cycle types: 2
1032 orbit reps of type ([1, 1, 1, 1, 1, 1, 1], [7], [7], [7])
1440 orbit reps of type ([7], [7], [7], [7])
